# Take Control of GPT : Use the API. Don't let the API use you

### My first openai API Project!!!

Here, I use the openai Python lib and some custom classes to handle processing prompts and responses. I store the results in a pandas dataframe, then format the code in a pandas 'style' object. I can use the dataframe and other objects (like the multiline string of prompts) to manage context for later prompts. Yes, context from prompts and answers. So much more to do with this. But this is a start.

## Setup

In [1]:
%%html
<style>
    /* Enable word wrap */
    .rendered_html table, .rendered_html th, .rendered_html tr, .rendered_html td {
      white-space: nowrap;
      overflow: hidden;
      text-overflow: ellipsis;
    }
</style>

### Imports

In [2]:
# ...
import openai
import os
import pprint
import sys
from pprint import pprint
import json
import pyarrow as pa
import pandas as pd

lib = os.path.abspath(os.path.join('..', '..', 'libs'))
sys.path.append(lib)

from MessageParser import MessageParser
from ResponseParser import ResponseParser
from PromptLogs import PromptLogs


### Config

In [3]:
# ...
# *******************************************************************************************************
# SET THE API KEY
# ================
# -- set the env variable OPENAI_API_KEY to your OpenAI API key       <<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<<
openai.api_key = os.getenv("OPENAI_API_KEY")
# *******************************************************************************************************

# Setup up pandas properties
pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.expand_frame_repr', False)  # Prevent line break in DataFrame
pd.set_option('display.max_colwidth', None)  # Display the full content of cells



## Main code

Prep the system prompts

In [4]:
sys_prompt_list = []
sys_prompt_1 = """You are a software expert."""
sys_prompt_2 = """Use. "```python" style for code blocks."""
sys_prompt_3 = """Include the name for a file where I would store individual code for functions or classes."""

sys_prompt_list.append(sys_prompt_1)
sys_prompt_list.append(sys_prompt_2)
sys_prompt_list.append(sys_prompt_3)

sys_prompt_string = "".join(sys_prompt_list)

# we'll be storing the individual parser logs here 

In [5]:
logs = PromptLogs()

Generate the message and make the call

In [6]:
message_list = f"""
system: {sys_prompt_string}
user: Write python code for fibonacci sequence. also write a function to zero pad a string to 10 chars.
"""

parser = MessageParser(message_list)
messages = parser.parse_messages()

# Get the response
# I'll abstract this away in the future
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)


Get a parser object, using a custom class.

In [7]:
parser = ResponseParser(response=response)

get a pandas dataframe of the logged prompts and responses

In [8]:
log_df = parser.get_df()
log_df

,id,created,model,usage,all_code,code_blocks,content
0,chatcmpl-73yJ2wZYTC7qQrdU6HtV09sQ2XjN0,1.681181e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;fib_seq&nbsp;=&nbsp;[0,&nbsp;1]<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;len(fib_seq)&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;next_fib&nbsp;=&nbsp;fib_seq[-1]&nbsp;+&nbsp;fib_seq[-2]<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fib_seq.append(next_fib)<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;fib_seq<br><br>print(fibonacci(10))<br>def&nbsp;zero_pad_str(s):<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;s.ljust(10,&nbsp;'0')<br><br>my_str&nbsp;=&nbsp;'1234'<br>my_padded_str&nbsp;=&nbsp;zero_pad_str(my_str)<br>print(my_padded_str)<br>","[\ndef fibonacci(n):\n fib_seq = [0, 1]\n while len(fib_seq) < n:\n next_fib = fib_seq[-1] + fib_seq[-2]\n fib_seq.append(next_fib)\n return fib_seq\n\nprint(fibonacci(10))\n, \ndef zero_pad_str(s):\n return s.ljust(10, '0')\n\nmy_str = '1234'\nmy_padded_str = zero_pad_str(my_str)\nprint(my_padded_str)\n]","Here is the Python code for Fibonacci sequence:\n\n```python\ndef fibonacci(n):\n fib_seq = [0, 1]\n while len(fib_seq) < n:\n next_fib = fib_seq[-1] + fib_seq[-2]\n fib_seq.append(next_fib)\n return fib_seq\n\nprint(fibonacci(10))\n```\n\nThis will print the first 10 numbers in the Fibonacci sequence `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.\n\nHere is the Python code for a function that zero pads a string to 10 chars:\n\n```python\ndef zero_pad_str(s):\n return s.ljust(10, '0')\n\nmy_str = '1234'\nmy_padded_str = zero_pad_str(my_str)\nprint(my_padded_str)\n```\n\nIf `my_str = '1234'`, this function will return a string `""1234000000""`."


In [9]:
# this is a PromptLogs object
logs.append(parser)

In [10]:
logs.get_df()

,created,model,usage,all_code,code_blocks,content
id,,,,,,
chatcmpl-73yJ2wZYTC7qQrdU6HtV09sQ2XjN0,1.681181e+09,gpt-3.5-turbo-0301,66.0,"def&nbsp;fibonacci(n):<br>&nbsp;&nbsp;&nbsp;&nbsp;fib_seq&nbsp;=&nbsp;[0,&nbsp;1]<br>&nbsp;&nbsp;&nbsp;&nbsp;while&nbsp;len(fib_seq)&nbsp;<&nbsp;n:<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;next_fib&nbsp;=&nbsp;fib_seq[-1]&nbsp;+&nbsp;fib_seq[-2]<br>&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;fib_seq.append(next_fib)<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;fib_seq<br><br>print(fibonacci(10))<br>def&nbsp;zero_pad_str(s):<br>&nbsp;&nbsp;&nbsp;&nbsp;return&nbsp;s.ljust(10,&nbsp;'0')<br><br>my_str&nbsp;=&nbsp;'1234'<br>my_padded_str&nbsp;=&nbsp;zero_pad_str(my_str)<br>print(my_padded_str)<br>","[\ndef fibonacci(n):\n fib_seq = [0, 1]\n while len(fib_seq) < n:\n next_fib = fib_seq[-1] + fib_seq[-2]\n fib_seq.append(next_fib)\n return fib_seq\n\nprint(fibonacci(10))\n, \ndef zero_pad_str(s):\n return s.ljust(10, '0')\n\nmy_str = '1234'\nmy_padded_str = zero_pad_str(my_str)\nprint(my_padded_str)\n]","Here is the Python code for Fibonacci sequence:\n\n```python\ndef fibonacci(n):\n fib_seq = [0, 1]\n while len(fib_seq) < n:\n next_fib = fib_seq[-1] + fib_seq[-2]\n fib_seq.append(next_fib)\n return fib_seq\n\nprint(fibonacci(10))\n```\n\nThis will print the first 10 numbers in the Fibonacci sequence `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34]`.\n\nHere is the Python code for a function that zero pads a string to 10 chars:\n\n```python\ndef zero_pad_str(s):\n return s.ljust(10, '0')\n\nmy_str = '1234'\nmy_padded_str = zero_pad_str(my_str)\nprint(my_padded_str)\n```\n\nIf `my_str = '1234'`, this function will return a string `""1234000000""`."


In [11]:
# A pandas style applies custom formatting to a DataFrame to make it easier to read and
# understand. It does not offer the full range of functionality that a DataFrame provides.
parser.styled_log

,id,created,model,usage,all_code
0,chatcmpl-73yJ2wZYTC7qQrdU6HtV09sQ2XjN0,1681181156.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci(n): fib_seq = [0, 1] while len(fib_seq) < n: next_fib = fib_seq[-1] + fib_seq[-2] fib_seq.append(next_fib) return fib_seqprint(fibonacci(10))def zero_pad_str(s): return s.ljust(10, '0')my_str = '1234'my_padded_str = zero_pad_str(my_str)print(my_padded_str)"


In [ ]:
logs.df

## And we make another exchange

In [12]:
message_list = f"""
system: {sys_prompt_string}
user: Write a python function that returns numbers 1-10 in a list. 
"""

message_parser = MessageParser(message_list)
messages = message_parser.parse_messages()
response = openai.ChatCompletion.create(model='gpt-3.5-turbo', temperature=1.3, max_tokens=300, messages = messages)
response_parser = ResponseParser(response=response)

logs.append(response_parser)

## And print the styled logs with the code from two separate prompt exchanges

In [13]:
logs.styled_log

,created,model,usage,all_code
id,,,,
nan,1681181156.000000,gpt-3.5-turbo-0301,66.000000,"def fibonacci(n): fib_seq = [0, 1] while len(fib_seq) < n: next_fib = fib_seq[-1] + fib_seq[-2] fib_seq.append(next_fib) return fib_seqprint(fibonacci(10))def zero_pad_str(s): return s.ljust(10, '0')my_str = '1234'my_padded_str = zero_pad_str(my_str)print(my_padded_str)"
chatcmpl-73yJC0F2Jw9VPNF5NmWCaE2AhK8Dt,1681181166.000000,gpt-3.5-turbo-0301,60.000000,"from number_functions import get_numbersresult = get_numbers()print(result)[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]"
